In [ ]:
from edge_sensor.sdr import airt
from channel_analysis import waveform
from iqwaveform import fourier

duration = 0.1
center_frequency_Hz = 3e9
analysis_bandwidth_Hz = 10e6
sample_rate_Hz = 15.36e6

fs_sdr, lo_offset, resample_kws = fourier.design_cola_frequency_shift(fs_base=125e6, fs_target=sample_rate_Hz, bw=analysis_bandwidth_Hz)

print('connecting...')
capture = airt.AirTCapture(frequency_Hz=center_frequency_Hz+2*lo_offset, sample_rate_Hz=fs_sdr, gain_dB=0)

try:
    print('acquiring...')
    raw_iq = capture.acquire(int(duration * fs_sdr))
    resample_kws['frequency_shift'] = 'right'
    iq = fourier.ola_filter(raw_iq, passband=(-analysis_bandwidth_Hz, analysis_bandwidth_Hz), **resample_kws)
finally:
    print('done')
    capture.close()

# # Option 2: No resample and shift
# print('connecting...')
# capture = airt.AirTCapture(frequency_Hz=center_frequency_Hz, sample_rate_Hz=sample_rate_Hz, gain_dB=0)

# try:
#     print('acquiring...')
#     iq = capture.acquire(int(duration * sample_rate_Hz))
# finally:
#     print('done')
#     capture.close()

In [ ]:
# the following could be loaded from e.g. json
from edge_sensor.analysis import from_spec

analysis_spec = {
    'power_time_series': {
        'detector_period': 10e-3,
        'detectors': ('rms', 'peak')
    },
    'cyclic_channel_power': {
        'cyclic_period': 10e-3,
        'detector_period': 1e-3 / 15 / 4,
        'detectors': ('rms', 'peak'),
        'cyclic_statistics': ('min', 'mean', 'max'),
    },
    'persistence_spectrum': {
        'window': 'flattop',
        'resolution': 15e3,
        'quantiles': [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 0.999, 1],
        'truncate': False
    },
    'amplitude_probability_distribution': {
        'power_low': -105,
        'power_high': -60,
        'power_count': 221,  # 0.25 dB resolution
    },
    # 'iq_waveform': {
    #     'start_time_sec': 0,
    #     'stop_time_sec': 100e-3
    # }
}

filter_spec = {}

dataset = from_spec(
    iq,
    sample_rate_Hz,
    analysis_bandwidth_Hz=analysis_bandwidth_Hz,
    filter_spec=filter_spec,
    analysis_spec=analysis_spec,
)

In [ ]:
import iqwaveform
from channel_analysis import figures
import matplotlib.pyplot as plt
from iqwaveform import powtodB

plt.style.use('iqwaveform.ieee_double_column')

# fig, ax = plt.subplots()
# pvt = dataset.power_time_series
# pvt.plot.line(x='time_elapsed', ax=ax)
# figures.label_axis(ax.xaxis, pvt, 'time_elapsed')
# figures.label_axis(ax.yaxis, pvt, tick_units=False)
# figures.label_legend(ax, pvt, 'power_detector')

# fig, ax = plt.subplots()
# apd = dataset.amplitude_probability_distribution
# apd.plot.line(x='channel_power', ax=ax)
# ax.set_yscale('gamma-qq', k=1, db_ordinal=True)
# figures.label_axis(ax.xaxis, apd, 'channel_power', tick_units=False)
# figures.label_axis(ax.yaxis, apd)

# fig, ax = plt.subplots()
# cyclic_power = dataset.cyclic_channel_power
# time = cyclic_power.cyclic_lag
# for i, detector in enumerate(cyclic_power.power_detector.values):
#     a = cyclic_power.sel(power_detector=detector)
#     ax.plot(time, powtodB(a.sel(cyclic_statistic='mean')), color=f'C{i}')

# for i, detector in enumerate(cyclic_power.power_detector.values):
#     a = cyclic_power.sel(power_detector=detector)
#     ax.fill_between(
#         time,
#         powtodB(a.sel(cyclic_statistic='min')),
#         powtodB(a.sel(cyclic_statistic='max')),
#         color=f'C{i}',
#         alpha=0.25,
#         lw=0,
#         rasterized=True,
#     )

# figures.label_axis(ax.xaxis, cyclic_power, 'cyclic_lag')
# figures.label_axis(ax.yaxis, cyclic_power, tick_units=False)
# figures.label_legend(ax, cyclic_power, 'power_detector')

fig, ax = plt.subplots()
spectrum = dataset.persistence_spectrum
spectrum.plot.line(x='baseband_frequency', ax=ax)
figures.label_axis(ax.xaxis, spectrum, 'baseband_frequency')
figures.label_axis(ax.yaxis, spectrum, tick_units=False)
figures.label_legend(ax, spectrum, 'persistence_statistic')

In [ ]:
spectrum.sel(persistence_statistic='0.5').idxmax()

In [ ]:
lo_offset*(2304/3125)